## Universidad Autonoma de Aguascalientes
## Departamento: Ciencias de la computacion
## Carrera: Ingenieria en Computacion Inteligente
## Curso: Machine y Deep Learning
## Maestro: Dr. Francisco Javier Luna Rosas
## Alumno: Guillermo González Lara (237864)
## Semestre: Enero_Junio del 2026

# Práctica No. 7: Análisis de Sentimientos con Redes Neuronales (MLPClassifier)

## Objetivo
En esta práctica implementaremos un modelo de análisis de sentimientos utilizando el algoritmo de **Perceptrón Multicapa (MLP)** de la librería Scikit-Learn. 

El objetivo es clasificar críticas de cine del dataset `movie_data.csv` en dos categorías:
* **Positiva (1)**
* **Negativa (0)**

Dado que las redes neuronales requieren entradas numéricas, utilizaremos técnicas de procesamiento de lenguaje natural (NLP) como la **Bolsa de Palabras (Bag of Words)** para transformar el texto de las críticas en vectores numéricos antes de entrenar la red.

### Paso 1: Importar las librerías necesarias

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Para mostrar todas las columnas si fuera necesario
pd.set_option('display.max_colwidth', 100)

### Paso 2: Cargar el dataset
Cargamos el archivo `movie_data.csv` que se encuentra en la carpeta raíz. Este dataset contiene dos columnas principales: `review` (el texto de la crítica) y `sentiment` (la etiqueta: 0 o 1).

In [4]:
# Carga del dataset asumiendo que está en la raíz
df = pd.read_csv('../movie_data.csv')

# Mostrar las primeras 5 filas para verificar la estructura
print("Primeras filas del dataset:")
display(df.head())

# Verificar la distribución de clases (cuántas positivas y negativas hay)
print("\nDistribución de etiquetas:")
print(df['sentiment'].value_counts())

Primeras filas del dataset:


,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, ...",1
1,OK... so... I really like Kris Kristofferson and his usual easy going delivery of lines in his m...,0
2,"***SPOILER*** Do not read this, if you think about watching that movie, although it would be a w...",0
3,hi for all the people who have seen this wonderful movie im sure thet you would have liked it as...,1
4,"I recently bought the DVD, forgetting just how much I hated the movie version of ""A Chorus Line....",0



Distribución de etiquetas:
sentiment
1    25000
0    25000
Name: count, dtype: int64


### Paso 3: Preprocesamiento y Vectorización de Texto
Las redes neuronales no pueden procesar texto crudo directamente. Necesitamos convertir las oraciones en números.

Utilizaremos `CountVectorizer` para crear una "Bolsa de Palabras". Esto convertirá cada crítica en un vector que cuenta la frecuencia de cada palabra.
* **stop_words='english'**: Eliminaremos palabras comunes en inglés (como "the", "is", "at") que no aportan mucho significado.
* **max_features=2000**: Limitaremos el vocabulario a las 2000 palabras más frecuentes para evitar que la red neuronal sea demasiado pesada y lenta para este ejercicio.

In [5]:
# Definimos las variables de entrada (X) y salida (y)
X_texto = df['review']
y = df['sentiment']

# Instanciamos el vectorizador
# Limitamos a 2000 palabras para agilizar el cómputo en esta práctica
vectorizer = CountVectorizer(stop_words='english', max_features=2000)

# Transformamos el texto a números
X = vectorizer.fit_transform(X_texto)

print(f"Dimensión de la matriz de características (X): {X.shape}")
print("Ejemplo: Las críticas ahora son representaciones numéricas dispersas.")

Dimensión de la matriz de características (X): (50000, 2000)
Ejemplo: Las críticas ahora son representaciones numéricas dispersas.


### Paso 4: División de datos en entrenamiento y prueba
Separamos los datos en dos conjuntos: 70% para entrenar la red neuronal y 30% para evaluar su desempeño con datos no vistos.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Datos de entrenamiento: {X_train.shape}")
print(f"Datos de prueba: {X_test.shape}")

Datos de entrenamiento: (35000, 2000)
Datos de prueba: (15000, 2000)


### Paso 5: Implementación y Entrenamiento del MLPClassifier
Configuramos la red neuronal. Usaremos:
* **hidden_layer_sizes=(100, 50)**: Dos capas ocultas, la primera con 100 neuronas y la segunda con 50.
* **activation='relu'**: Función de activación estándar para capas ocultas.
* **max_iter=500**: Aumentamos las iteraciones máximas para asegurar que el modelo converja.

In [7]:
# Instanciar el clasificador MLP
mlp = MLPClassifier(hidden_layer_sizes=(100, 50), 
                    activation='relu', 
                    solver='adam', 
                    max_iter=500, 
                    random_state=42,
                    verbose=True) # verbose=True para ver el progreso del entrenamiento

# Entrenar el modelo
mlp.fit(X_train, y_train)

print("\nEntrenamiento finalizado.")

Iteration 1, loss = 0.37922949
Iteration 2, loss = 0.26496224
Iteration 3, loss = 0.21221447
Iteration 4, loss = 0.13986347
Iteration 5, loss = 0.07186221
Iteration 6, loss = 0.02966865
Iteration 7, loss = 0.01082870
Iteration 8, loss = 0.00401549
Iteration 9, loss = 0.00191267
Iteration 10, loss = 0.00123783
Iteration 11, loss = 0.00083624
Iteration 12, loss = 0.00059205
Iteration 13, loss = 0.00048593
Iteration 14, loss = 0.00041904
Iteration 15, loss = 0.00037181
Iteration 16, loss = 0.00033703
Iteration 17, loss = 0.00031078
Iteration 18, loss = 0.00029069
Iteration 19, loss = 0.00027483
Iteration 20, loss = 0.00026209
Iteration 21, loss = 0.00025183
Iteration 22, loss = 0.00024345
Iteration 23, loss = 0.00023648
Iteration 24, loss = 0.00023069
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.

Entrenamiento finalizado.


### Paso 6: Evaluación del Modelo
Realizamos predicciones sobre el conjunto de prueba (`X_test`) y evaluamos la precisión utilizando la matriz de confusión y el reporte de clasificación.

In [8]:
# Predicción
y_pred = mlp.predict(X_test)

# Matriz de Confusión
print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))

# Reporte de Clasificación
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Precisión Global
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.4f}")

Matriz de Confusión:
[[6366 1130]
 [1015 6489]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.86      0.85      0.86      7496
           1       0.85      0.86      0.86      7504

    accuracy                           0.86     15000
   macro avg       0.86      0.86      0.86     15000
weighted avg       0.86      0.86      0.86     15000

Precisión del modelo: 0.8570


## Conclusiones

En esta práctica número 7, aplicamos una red neuronal de tipo Perceptrón Multicapa (MLP) para una tarea de Procesamiento de Lenguaje Natural (NLP). Logramos transformar texto no estructurado (críticas de cine) en datos numéricos utilizando `CountVectorizer`.

El modelo fue capaz de aprender patrones en las palabras asociadas a críticas positivas y negativas. A pesar de limitar el vocabulario a 2000 palabras por razones de eficiencia, el `MLPClassifier` demostró ser una herramienta robusta para problemas de clasificación binaria con datos de alta dimensionalidad.

### Referencias
* Scikit-learn: Machine Learning in Python. URL: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
* Pang, B., & Lee, L. (2004). A sentimental education: Sentiment analysis using subjectivity summarization based on minimum cuts. (Dataset source).